In [ ]:
# default_exp core

# CircuitPython_pico_pi_controller

> The `pico_pi_controller` module contains CircuitPython classes to manage multiple Raspberry Pi SBCs from a Raspberry Pi Pico, using i2c.

## About

The classes in this module inherit from Adafruit's `busio` & `adafruit_bus_device` I2C classes for CircuitPython. `busio` is distributed with [CircuitPython](https://circuitpython.org), and `adafruit_bus_device` is distributed with the [CircuitPython Library Bundle](https://circuitpython.org/libraries).

Broadcom SoC hardware I2C is utilized both for the controlling MCU (the Pico) and the controlled SBCs (Raspberry Pi 4/3+/Zero). I2C baudrate is limited from the controller (the Pico) to reduce transmit/receive errors.

Controller programs may be written with hard-coded I2C peripheral addresses of connected SBCs (called 'devices' by these classes). Or, autodiscovery may be used so that devices may be added without changing software on the controller.

For example:

<code>
from pico_pi_controller import *
    
mycontroller = PPController()

mycontroller.addDevice('0x13')
mycontroller.listdevices()
</code>

This example will initialize I2C on the controller using the board defaults (I2C bus 1), retrieve host information & statistics from the SBC at I2C address 0x13, then return a list of device information.

## Module

In [ ]:
#hide
try:
    from nbdev.showdoc import *
except:
    pass
    """CircuitPython kernel has no nbdev"""

In [ ]:
#export
from sys import byteorder, modules
from time import sleep, struct_time
import board
import busio
from adafruit_bus_device import i2c_device
try:
    from rtc import RTC
except:
    pass
try:
    from adafruit_datetime import datetime
except:
    from datetime import datetime
try:
    from adafruit_itertools.adafruit_itertools import chain
except:
    from itertools import chain
try:
    import adafruit_logging as logging
except:
    import logging
    

try:
    logger = logging.getLogger('PPC')
except:
    logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

try:
    logging.basicConfig(level = logging.DEBUG)
except:
    pass

IDENTITY = bytearray(([ord(c) for c in list('ppc-daemon')]))
"""identifier string used by RPi devices"""

IDF = bytearray(); IDF.append(ord('I')) # request to send identification
HOS = bytearray(); HOS.append(ord('H')) # request to send hostname
LOD = bytearray(); LOD.append(ord('L')) # request to send load
TIM = bytearray(); TIM.append(ord('T')) # request to send datetime
PEN = bytearray(); PEN.append(ord('P')) # request to send MCU pin connected to RPi PEN

In [ ]:
#export
class PPDevice():
    """Represents an I2C peripheral device known to a `PPController`
    and stores data from those peripherals identified as PPC devices."""
    def __init__(self, controller=None, device_address=None, *argv, **kwargs):
        self.controller = controller
        self.device_address = device_address
        self.device=None
        
        self.retries = 0
        self.retry_max = 10
        """retry count before I2CDevice is considered 'other', i.e. not a PPC device."""
        
        """All data below are received *from* the PPC device:"""
        
        self.bosmang = None
        self.UART_RX = None
        """MCU GPIO RX for passthru from bosmang console TX"""
        self.UART_TX = None
        """MCU GPIO TX for passthru from bosmang console RX"""
        self.pen = None
        """MCU GPIO connected to RPi PEN pin"""

        
        self.hostname = None
        self.load = ""
        self.datetime = None
        """to send datetime as bosmang, and used to check for datetime skew
           on other devices."""
        
    def getHostname(self): 
        return self.hostname

In [ ]:
#export
class PPController():
    """Represents one of the system's I2C busses and tracks which I2C
    peripherals are `PPDevice`s."""
    def __init__(self, scl=board.SCL, sda=board.SDA, frequency=4800, timeout=10000):
        self.i2c = None
        self.scl = scl
        self.sda = sda
        self.frequency=frequency
        self.timeout=timeout
 
        self.bosmang = None
        """PPDevice hostname selected to recieve datetime & control instructions from,
           have UART connected for passthru, etc. Default is `None`."""
        
        self.devices = []
        self.noident = []
        self.othrdev = []
        
        self.datetime = None
        """future use for receiving datetime from bosmang"""

def scanForNew(controller):
    if controller.i2c.try_lock():
        logger.info("PPC: scanning I2C bus for PPController at SCL/SDA "
                    +str(controller.scl)+"/"+str(controller.sda))
    else:
        logger.info("PPC: I2C lock not avilable")
        return False
    scan_addrs=controller.i2c.scan()
    #controller.i2c.unlock()
    for addr in scan_addrs:
        if not any(d.device_address == addr for d in chain(controller.devices,controller.noident,controller.othrdev)):  
            controller.noident.append(PPDevice(i2c=controller.i2c,device_address=addr))
            with controller.i2c as i2c:
                controller.noident[-1].device=i2c_device.I2CDevice(i2c,device_address=addr)
            logger.info("PPC: added I2C peripheral object "+str(hex(addr))+" to PPController at SCL/SDA "
                        +str(controller.scl)+"/"+str(controller.sda))
    controller.i2c.unlock()
    return True
                
def identifyNew(controller):
    for peripheral in controller.noident:
        with peripheral.device as unident:
            msg = bytearray(len(IDENTITY))
            try:
                unident.write_then_readinto(IDF,msg)
            except:
                pass
            if msg == IDENTITY:
                self.devices.append(self.noident.pop(self.noident.index(peripheral)))
            else:
                peripheral.retries += 1
                if peripheral.retries >= peripheral.retries_max:
                    self.othrdev.append(self.noident.pop(self.noident.index(peripheral)))
                
    
def autoAddDevices(controller):
    controller.i2c.scanForNew()
    identifyNew(controller)
                
def listdevices(controller):
    return controller.devices


## Debugging

```python
con=PPController()
con.i2c=busio.I2C(scl=con.scl,sda=con.sda,frequency=con.frequency)
scanForNew(con)
```

In [ ]:
try:
    from CircuitPython_pico_pi_controller import *
except:
    pass

con=PPController()
con.i2c=busio.I2C(scl=con.scl,sda=con.sda,frequency=con.frequency)
scanForNew(con)
#identifyNew(con)

#dev=PPDevice(con.i2c,0x13)
#print(dir(dev))

INFO:PPC:PPC: scanning I2C bus for PPController at SCL/SDA 3/2


## Appendix

In [ ]:
try:
    from nbdev.export import notebook2script
    notebook2script()
except:
    pass